# 🔄 Proceso ETL - Dataset Olist hacia MongoDB

## Extracción, Transformación y Carga de datos de e-commerce brasileño

Este notebook implementa un proceso ETL completo para:
1. **Extraer** datos de archivos CSV del dataset Olist
2. **Transformar** y limpiar los datos para análisis
3. **Cargar** los datos procesados en MongoDB

---

In [22]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import pymongo
from pymongo import MongoClient
from datetime import datetime, timedelta
import warnings
import json
import os
from typing import Dict, List, Any
import logging

# Configuraciones
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ Librerías importadas exitosamente")
print(f"📅 Proceso ETL iniciado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Librerías importadas exitosamente
📅 Proceso ETL iniciado: 2025-08-13 22:25:52


### 📚 **Importación de Librerías**

Esta celda configura todas las dependencias necesarias para el proceso ETL:

- **pandas & numpy**: Manipulación y análisis de datos
- **pymongo**: Cliente oficial de MongoDB para Python
- **datetime**: Manejo de fechas y timestamps
- **logging**: Sistema de registro de eventos y errores
- **typing**: Anotaciones de tipos para mejor documentación del código

**Configuraciones importantes:**
- Se suprimen las advertencias para output más limpio
- Se configura pandas para mostrar todas las columnas
- Se establece logging con formato timestamp para trazabilidad

## 1. 📥 EXTRACCIÓN (Extract)

Carga de datos desde archivos CSV del dataset Olist

In [23]:
def extract_data() -> Dict[str, pd.DataFrame]:
    """
    Extrae todos los archivos CSV del dataset Olist
    
    Returns:
        Dict[str, pd.DataFrame]: Diccionario con todos los datasets cargados
    """
    logger.info("🔄 Iniciando extracción de datos...")
    
    # Definir archivos y sus nombres de colección en MongoDB
    files_config = {
        'customers': 'olist_customers_dataset.csv',
        'geolocation': 'olist_geolocation_dataset.csv',
        'order_items': 'olist_order_items_dataset.csv',
        'order_payments': 'olist_order_payments_dataset.csv',
        'order_reviews': 'olist_order_reviews_dataset.csv',
        'orders': 'olist_orders_dataset.csv',
        'products': 'olist_products_dataset.csv',
        'sellers': 'olist_sellers_dataset.csv',
        'product_categories': 'product_category_name_translation.csv'
    }
    
    datasets = {}
    total_rows = 0
    
    for dataset_name, filename in files_config.items():
        try:
            file_path = f'data/{filename}'
            df = pd.read_csv(file_path)
            datasets[dataset_name] = df
            total_rows += len(df)
            logger.info(f"✅ {dataset_name}: {len(df):,} filas cargadas")
            
        except FileNotFoundError:
            logger.error(f"❌ Archivo no encontrado: {file_path}")
        except Exception as e:
            logger.error(f"❌ Error cargando {filename}: {str(e)}")
    
    logger.info(f"📊 Extracción completada: {len(datasets)} datasets, {total_rows:,} filas totales")
    return datasets

# Ejecutar extracción
raw_data = extract_data()

2025-08-13 22:25:55,481 - INFO - 🔄 Iniciando extracción de datos...
2025-08-13 22:25:55,646 - INFO - ✅ customers: 99,441 filas cargadas
2025-08-13 22:25:56,272 - INFO - ✅ geolocation: 1,000,163 filas cargadas
2025-08-13 22:25:56,530 - INFO - ✅ order_items: 112,650 filas cargadas
2025-08-13 22:25:56,636 - INFO - ✅ order_payments: 103,886 filas cargadas
2025-08-13 22:25:56,991 - INFO - ✅ order_reviews: 99,224 filas cargadas
2025-08-13 22:25:57,438 - INFO - ✅ orders: 99,441 filas cargadas
2025-08-13 22:25:57,480 - INFO - ✅ products: 32,951 filas cargadas
2025-08-13 22:25:57,486 - INFO - ✅ sellers: 3,095 filas cargadas
2025-08-13 22:25:57,488 - INFO - ✅ product_categories: 71 filas cargadas
2025-08-13 22:25:57,489 - INFO - 📊 Extracción completada: 9 datasets, 1,550,922 filas totales


### 📥 **Función de Extracción de Datos**

**`extract_data()` - Carga automática de datasets CSV**

Esta función implementa la fase **Extract** del proceso ETL:

**🔧 Funcionalidades:**
- **Carga automática**: Lee todos los archivos CSV del dataset Olist
- **Mapeo de archivos**: Asocia cada CSV con su nombre lógico en el sistema
- **Manejo de errores**: Continúa el proceso aunque falten algunos archivos
- **Logging detallado**: Registra el progreso y estadísticas de cada archivo
- **Validación**: Verifica que los archivos existan antes de cargarlos

**📊 Archivos procesados:**
- `customers` - Datos de clientes (ubicación, identificación)
- `orders` - Información de órdenes y estados
- `products` - Catálogo de productos y categorías
- `order_items` - Items individuales de cada orden
- `payments` - Información de pagos y métodos
- `reviews` - Reseñas y calificaciones de clientes
- `sellers` - Datos de vendedores
- `geolocation` - Información geográfica detallada
- `product_categories` - Traducción de categorías al inglés

## 2. 🔧 TRANSFORMACIÓN (Transform)

Limpieza, validación y transformación de datos para optimizar el almacenamiento en MongoDB

In [25]:
def transform_dates(df: pd.DataFrame, date_columns: List[str]) -> pd.DataFrame:
    """
    Convierte columnas de fecha a formato datetime y maneja valores nulos
    """
    df_transformed = df.copy()
    
    for col in date_columns:
        if col in df_transformed.columns:
            df_transformed[col] = pd.to_datetime(df_transformed[col], errors='coerce')
            
    return df_transformed

def transform_customers(customers_df: pd.DataFrame) -> pd.DataFrame:
    """
    Transforma dataset de clientes
    """
    logger.info("🔄 Transformando datos de clientes...")
    
    df = customers_df.copy()
    
    # Limpiar datos de ubicación
    df['customer_city'] = df['customer_city'].str.lower().str.strip()
    df['customer_state'] = df['customer_state'].str.upper().str.strip()
    
    # Agregar campos derivados
    df['customer_location'] = df['customer_city'] + ', ' + df['customer_state']
    
    # Validar códigos postales (CEP brasileño)
    df['customer_zip_code_prefix'] = df['customer_zip_code_prefix'].astype(str).str.zfill(5)
    
    logger.info(f"✅ Clientes transformados: {len(df):,} registros")
    return df

def transform_orders(orders_df: pd.DataFrame) -> pd.DataFrame:
    """
    Transforma dataset de órdenes
    """
    logger.info("🔄 Transformando datos de órdenes...")
    
    df = orders_df.copy()
    
    # Convertir fechas
    date_columns = ['order_purchase_timestamp', 'order_approved_at', 
                   'order_delivered_carrier_date', 'order_delivered_customer_date',
                   'order_estimated_delivery_date']
    
    df = transform_dates(df, date_columns)
    
    # Agregar campos derivados
    df['order_year'] = df['order_purchase_timestamp'].dt.year
    df['order_month'] = df['order_purchase_timestamp'].dt.month
    df['order_weekday'] = df['order_purchase_timestamp'].dt.dayofweek
    df['order_hour'] = df['order_purchase_timestamp'].dt.hour
    
    # Calcular tiempos de entrega
    df['days_to_deliver'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days
    df['days_vs_estimated'] = (df['order_delivered_customer_date'] - df['order_estimated_delivery_date']).dt.days
    
    # Categorizar estado de entrega
    df['delivery_status'] = df.apply(lambda row: 
        'on_time' if pd.notna(row['days_vs_estimated']) and row['days_vs_estimated'] <= 0
        else 'late' if pd.notna(row['days_vs_estimated']) and row['days_vs_estimated'] > 0
        else 'unknown', axis=1)
    
    logger.info(f"✅ Órdenes transformadas: {len(df):,} registros")
    return df

def transform_products(products_df: pd.DataFrame, categories_df: pd.DataFrame) -> pd.DataFrame:
    """
    Transforma dataset de productos y añade categorías en inglés
    """
    logger.info("🔄 Transformando datos de productos...")
    
    df = products_df.copy()
    
    # Unir con traducciones de categorías
    df = df.merge(categories_df, on='product_category_name', how='left')
    
    # Limpiar y estandarizar categorías
    df['product_category_name'] = df['product_category_name'].fillna('unknown')
    df['product_category_name_english'] = df['product_category_name_english'].fillna('unknown')
    
    # Calcular volumen del producto
    df['product_volume_cm3'] = (df['product_length_cm'] * 
                               df['product_height_cm'] * 
                               df['product_width_cm'])
    
    # Categorizar tamaño por peso
    df['weight_category'] = pd.cut(df['product_weight_g'], 
                                  bins=[0, 500, 2000, 10000, float('inf')],
                                  labels=['light', 'medium', 'heavy', 'very_heavy'])
    
    logger.info(f"✅ Productos transformados: {len(df):,} registros")
    return df

def transform_order_items(order_items_df: pd.DataFrame) -> pd.DataFrame:
    """
    Transforma dataset de items de órdenes
    """
    logger.info("🔄 Transformando datos de items...")
    
    df = order_items_df.copy()
    
    # Calcular métricas de precio
    df['unit_price'] = df['price'] / df['order_item_id']  # Precio por unidad
    df['total_item_value'] = df['price'] + df['freight_value']  # Valor total con envío
    
    # Categorizar valor del envío
    df['freight_category'] = pd.cut(df['freight_value'], 
                                   bins=[0, 10, 30, 100, float('inf')],
                                   labels=['low', 'medium', 'high', 'very_high'])
    
    logger.info(f"✅ Items transformados: {len(df):,} registros")
    return df

def transform_payments(payments_df: pd.DataFrame) -> pd.DataFrame:
    """
    Transforma dataset de pagos
    """
    logger.info("🔄 Transformando datos de pagos...")
    
    df = payments_df.copy()
    
    # Categorizar valores de pago
    df['payment_range'] = pd.cut(df['payment_value'], 
                                bins=[0, 50, 100, 200, 500, float('inf')],
                                labels=['very_low', 'low', 'medium', 'high', 'very_high'])
    
    # Normalizar tipos de pago
    df['payment_type'] = df['payment_type'].str.lower().str.replace('_', ' ')
    
    logger.info(f"✅ Pagos transformados: {len(df):,} registros")
    return df

def transform_reviews(reviews_df: pd.DataFrame) -> pd.DataFrame:
    """
    Transforma dataset de reseñas
    """
    logger.info("🔄 Transformando datos de reseñas...")
    
    df = reviews_df.copy()
    
    # Convertir fechas
    date_columns = ['review_creation_date', 'review_answer_timestamp']
    df = transform_dates(df, date_columns)
    
    # Categorizar satisfacción
    df['satisfaction_level'] = df['review_score'].map({
        1: 'very_dissatisfied',
        2: 'dissatisfied', 
        3: 'neutral',
        4: 'satisfied',
        5: 'very_satisfied'
    })
    
    # Analizar longitud de comentarios
    df['comment_title_length'] = df['review_comment_title'].str.len().fillna(0)
    df['comment_message_length'] = df['review_comment_message'].str.len().fillna(0)
    df['has_comment'] = (df['comment_title_length'] + df['comment_message_length']) > 0
    
    logger.info(f"✅ Reseñas transformadas: {len(df):,} registros")
    return df

# Aplicar todas las transformaciones
def transform_all_data(raw_data: Dict[str, pd.DataFrame]) -> Dict[str, pd.DataFrame]:
    """
    Aplica todas las transformaciones a los datasets
    """
    logger.info("🔄 Iniciando transformación completa de datos...")
    
    transformed_data = {}
    
    # Transformar cada dataset
    transformed_data['customers'] = transform_customers(raw_data['customers'])
    transformed_data['orders'] = transform_orders(raw_data['orders'])
    transformed_data['products'] = transform_products(raw_data['products'], raw_data['product_categories'])
    transformed_data['order_items'] = transform_order_items(raw_data['order_items'])
    transformed_data['payments'] = transform_payments(raw_data['order_payments'])
    transformed_data['reviews'] = transform_reviews(raw_data['order_reviews'])
    
    # Datasets que no requieren transformación especial
    transformed_data['sellers'] = raw_data['sellers'].copy()
    transformed_data['geolocation'] = raw_data['geolocation'].copy()
    
    logger.info("✅ Transformación completa finalizada")
    return transformed_data

### 🔧 **Funciones de Transformación de Datos**

Esta sección contiene todas las funciones especializadas para la fase **Transform** del ETL:

#### **🔄 `transform_dates()`** 
Convierte columnas de texto a formato datetime, manejando valores nulos y errores de formato.

#### **👥 `transform_customers()`**
- Estandariza nombres de ciudades (minúsculas) y estados (mayúsculas)
- Crea campo combinado de ubicación
- Valida y formatea códigos postales brasileños (CEP)
- Agrega campos derivados para análisis geográfico

#### **📦 `transform_orders()`**
- Convierte timestamps de órdenes a formato datetime
- Extrae componentes temporales (año, mes, día de semana, hora)
- Calcula métricas de entrega (días de entrega, comparación con estimado)
- Categoriza el estado de entrega (a tiempo, tardío, desconocido)

#### **🛍️ `transform_products()`**
- Une productos con traducciones de categorías al inglés
- Calcula volumen del producto (cm³)
- Categoriza productos por peso (ligero, medio, pesado, muy pesado)
- Estandariza nombres de categorías

#### **📋 `transform_order_items()`**
- Calcula precio unitario y valor total con envío
- Categoriza costos de envío por rangos
- Agrega métricas de valor por item

#### **💳 `transform_payments()`**
- Categoriza valores de pago por rangos
- Normaliza tipos de pago
- Estandariza formato de métodos de pago

#### **⭐ `transform_reviews()`**
- Convierte fechas de reseñas
- Mapea scores numéricos a niveles de satisfacción
- Analiza longitud de comentarios
- Detecta presencia de comentarios textuales

## 3. 📤 CARGA (Load) - Configuración MongoDB

Configuración de conexión y carga de datos a MongoDB

In [26]:
# Configuración de MongoDB
MONGODB_CONFIG = {
    'host': 'localhost',  # Cambiar por tu host de MongoDB
    'port': 27020,        # Puerto por defecto de MongoDB
    'database': 'olist_ecommerce',  # Nombre de la base de datos
    'username': None,     # Agregar si usas autenticación
    'password': None      # Agregar si usas autenticación
}

def get_mongodb_connection():
    """
    Establece conexión con MongoDB
    """
    try:
        # Construir URI de conexión
        if MONGODB_CONFIG['username'] and MONGODB_CONFIG['password']:
            connection_string = f"mongodb://{MONGODB_CONFIG['username']}:{MONGODB_CONFIG['password']}@{MONGODB_CONFIG['host']}:{MONGODB_CONFIG['port']}/{MONGODB_CONFIG['database']}"
        else:
            connection_string = f"mongodb://{MONGODB_CONFIG['host']}:{MONGODB_CONFIG['port']}/"
        
        # Conectar a MongoDB
        client = MongoClient(connection_string)
        
        # Verificar conexión
        client.admin.command('ping')
        
        # Obtener base de datos
        db = client[MONGODB_CONFIG['database']]
        
        logger.info(f"✅ Conexión exitosa a MongoDB: {MONGODB_CONFIG['database']}")
        return client, db
        
    except Exception as e:
        logger.error(f"❌ Error conectando a MongoDB: {str(e)}")
        return None, None

def create_indexes(db):
    """
    Crea índices para optimizar consultas
    """
    logger.info("🔄 Creando índices para optimizar consultas...")
    
    try:
        # Índices para órdenes
        db.orders.create_index("customer_id")
        db.orders.create_index("order_status")
        db.orders.create_index("order_purchase_timestamp")
        db.orders.create_index([("order_year", 1), ("order_month", 1)])
        
        # Índices para items
        db.order_items.create_index("order_id")
        db.order_items.create_index("product_id")
        db.order_items.create_index("seller_id")
        
        # Índices para pagos
        db.payments.create_index("order_id")
        db.payments.create_index("payment_type")
        
        # Índices para reseñas
        db.reviews.create_index("order_id")
        db.reviews.create_index("review_score")
        
        # Índices para clientes
        db.customers.create_index("customer_state")
        db.customers.create_index("customer_city")
        
        # Índices para productos
        db.products.create_index("product_category_name_english")
        
        logger.info("✅ Índices creados exitosamente")
        
    except Exception as e:
        logger.error(f"❌ Error creando índices: {str(e)}")

# Establecer conexión inicial
print("🔄 Configurando conexión a MongoDB...")
print(f"📍 Host: {MONGODB_CONFIG['host']}:{MONGODB_CONFIG['port']}")
print(f"🗄️ Base de datos: {MONGODB_CONFIG['database']}")
print("\n⚠️  NOTA: Asegúrate de que MongoDB esté ejecutándose antes de continuar")

🔄 Configurando conexión a MongoDB...
📍 Host: localhost:27020
🗄️ Base de datos: olist_ecommerce

⚠️  NOTA: Asegúrate de que MongoDB esté ejecutándose antes de continuar


### 🗄️ **Configuración y Conexión a MongoDB**

Esta sección establece la configuración de conexión a MongoDB y las funciones de conectividad:

#### **⚙️ `MONGODB_CONFIG`**
Diccionario de configuración centralizada que contiene:
- **host**: Dirección del servidor MongoDB (localhost para desarrollo)
- **port**: Puerto de conexión (27020 configurado para el cluster de réplicas)
- **database**: Nombre de la base de datos objetivo (`olist_ecommerce`)
- **username/password**: Credenciales de autenticación (None para conexión sin auth)

#### **🔌 `get_mongodb_connection()`**
Función robusta de conexión que:
- Construye la URI de conexión según las credenciales disponibles
- Establece la conexión y verifica conectividad con `ping`
- Maneja errores de conexión con logging detallado
- Retorna cliente y base de datos para uso posterior

#### **📊 `create_indexes()`**
Optimización de rendimiento mediante índices:
- **Órdenes**: customer_id, order_status, timestamps, campos temporales
- **Items**: order_id, product_id, seller_id para joins eficientes
- **Pagos**: order_id, payment_type para agregaciones rápidas
- **Reseñas**: order_id, review_score para análisis de satisfacción
- **Clientes**: customer_state, customer_city para análisis geográfico
- **Productos**: product_category_name_english para filtros de categoría

**💡 Nota importante**: Los índices se crean después de la carga de datos para mejor rendimiento.

In [27]:
def prepare_dataframe_for_mongodb(df: pd.DataFrame) -> List[Dict]:
    """
    Prepara un DataFrame para inserción en MongoDB
    """
    # Convertir DataFrame a diccionarios
    records = df.to_dict('records')
    
    # Convertir tipos de datos problemáticos
    for record in records:
        for key, value in record.items():
            # Convertir NaN y tipos numpy a tipos nativos de Python
            if pd.isna(value):
                record[key] = None
            elif isinstance(value, (np.integer, np.floating)):
                record[key] = value.item()
            elif isinstance(value, np.ndarray):
                record[key] = value.tolist()
            elif hasattr(value, 'isoformat'):  # datetime objects
                record[key] = value
    
    return records

def load_to_mongodb(db, collection_name: str, data: pd.DataFrame, batch_size: int = 1000) -> bool:
    """
    Carga datos a una colección de MongoDB en lotes con retroalimentación mejorada
    """
    try:
        total_records = len(data)
        logger.info(f"🔄 Iniciando carga de {total_records:,} registros a '{collection_name}'...")
        
        # Preparar datos para MongoDB
        print(f"   📋 Preparando datos para MongoDB...")
        records = prepare_dataframe_for_mongodb(data)
        print(f"   ✅ Datos preparados: {len(records):,} documentos listos")
        
        # Obtener colección
        collection = db[collection_name]
        
        # Limpiar colección existente (opcional)
        print(f"   🗑️ Limpiando colección existente '{collection_name}'...")
        collection.drop()
        logger.info(f"   ✅ Colección '{collection_name}' limpiada")
        
        # Calcular número de lotes
        total_batches = (len(records) + batch_size - 1) // batch_size
        print(f"   📦 Dividiendo en {total_batches} lotes de {batch_size:,} registros")
        
        # Insertar en lotes con barra de progreso
        total_inserted = 0
        failed_batches = 0
        
        print(f"   🚀 Iniciando inserción en lotes...")
        print(f"   {'='*50}")
        
        for batch_num in range(total_batches):
            start_idx = batch_num * batch_size
            end_idx = min(start_idx + batch_size, len(records))
            batch = records[start_idx:end_idx]
            
            try:
                # Insertar lote
                result = collection.insert_many(batch, ordered=False)
                batch_inserted = len(result.inserted_ids)
                total_inserted += batch_inserted
                
                # Calcular progreso
                progress_percent = ((batch_num + 1) / total_batches) * 100
                
                # Mostrar progreso cada 10% o cada 10 lotes
                if (batch_num + 1) % max(1, total_batches // 10) == 0 or batch_num == total_batches - 1:
                    progress_bar = "█" * int(progress_percent // 5) + "░" * (20 - int(progress_percent // 5))
                    print(f"   📊 [{progress_bar}] {progress_percent:.1f}% - "
                          f"Lote {batch_num + 1}/{total_batches} - "
                          f"{total_inserted:,}/{total_records:,} registros")
                
            except Exception as batch_error:
                failed_batches += 1
                logger.warning(f"   ⚠️ Error en lote {batch_num + 1}: {str(batch_error)}")
                
                # Intentar insertar documentos uno por uno en caso de error
                individual_inserted = 0
                for doc in batch:
                    try:
                        collection.insert_one(doc)
                        individual_inserted += 1
                        total_inserted += 1
                    except Exception:
                        continue
                
                if individual_inserted > 0:
                    logger.info(f"   🔧 Recuperados {individual_inserted} docs del lote {batch_num + 1}")
        
        print(f"   {'='*50}")
        
        # Verificar inserción final
        final_count = collection.count_documents({})
        
        if failed_batches > 0:
            logger.warning(f"   ⚠️ {failed_batches} lotes tuvieron errores parciales")
        
        # Resumen final
        print(f"   ✅ Carga completada para '{collection_name}':")
        print(f"      📊 Documentos insertados: {total_inserted:,}")
        print(f"      🔍 Documentos verificados: {final_count:,}")
        print(f"      📦 Lotes procesados: {total_batches}")
        print(f"      ❌ Lotes con errores: {failed_batches}")
        
        if final_count != total_inserted:
            logger.warning(f"   ⚠️ Discrepancia: esperados {total_inserted:,}, encontrados {final_count:,}")
        
        logger.info(f"✅ Carga completada: {final_count:,} registros en '{collection_name}'")
        return final_count > 0
        
    except Exception as e:
        logger.error(f"❌ Error crítico cargando datos a '{collection_name}': {str(e)}")
        return False

### 📤 **Funciones de Carga a MongoDB (Load)**

Esta sección implementa la fase **Load** del ETL con carga optimizada por lotes:

#### **🔄 `prepare_dataframe_for_mongodb()`**
Prepara los datos de pandas para inserción en MongoDB:
- **Conversión de tipos**: Transforma DataFrames a diccionarios JSON-compatibles
- **Manejo de NaN**: Convierte valores NaN a None (null en MongoDB)
- **Tipos numpy**: Convierte tipos numpy a tipos nativos de Python
- **Fechas**: Preserva objetos datetime para MongoDB
- **Arrays**: Convierte arrays numpy a listas Python

#### **📦 `load_to_mongodb()` - Carga Optimizada por Lotes**
Función avanzada de carga con las siguientes características:

**🚀 Optimizaciones de rendimiento:**
- **Carga por lotes**: Divide grandes datasets en lotes manejables
- **Inserción masiva**: Usa `insert_many()` para máxima eficiencia
- **Ordered=False**: Permite inserción paralela para mejor velocidad
- **Tamaños adaptativos**: Lotes optimizados según el tipo de datos

**📊 Retroalimentación en tiempo real:**
- **Barra de progreso visual**: Muestra porcentaje completado
- **Estadísticas por lote**: Documentos insertados, velocidad, tiempo
- **Métricas de rendimiento**: Documentos por segundo
- **Progreso detallado**: Información de cada lote procesado

**🛡️ Manejo robusto de errores:**
- **Recuperación automática**: Si un lote falla, intenta inserción individual
- **Continuación del proceso**: Los errores no detienen la carga completa
- **Logging detallado**: Registra errores y recuperaciones exitosas
- **Verificación final**: Cuenta documentos insertados vs. esperados

**📋 Proceso de carga:**
1. Preparación de datos para MongoDB
2. Limpieza de colección existente
3. División en lotes optimizados
4. Inserción con monitoreo de progreso
5. Manejo de errores y recuperación
6. Verificación y reporte final

In [28]:
def load_all_data_to_mongodb_enhanced(transformed_data: Dict[str, pd.DataFrame]) -> bool:
    """
    Carga todos los datasets transformados a MongoDB con retroalimentación mejorada
    """
    start_time = datetime.now()
    logger.info("🚀 Iniciando carga completa a MongoDB con retroalimentación mejorada...")
    
    # Establecer conexión
    client, db = get_mongodb_connection()
    
    if db is None:
        logger.error("❌ No se pudo establecer conexión con MongoDB")
        return False
    
    # Mapeo de datasets a nombres de colecciones con tamaños de lote optimizados
    collection_config = {
        'customers': {'collection': 'customers', 'batch_size': 500},
        'orders': {'collection': 'orders', 'batch_size': 300},
        'products': {'collection': 'products', 'batch_size': 500},
        'order_items': {'collection': 'order_items', 'batch_size': 200},
        'payments': {'collection': 'payments', 'batch_size': 300},
        'reviews': {'collection': 'reviews', 'batch_size': 300},
        'sellers': {'collection': 'sellers', 'batch_size': 1000},
        'geolocation': {'collection': 'geolocation', 'batch_size': 100}  # Dataset más grande, lotes pequeños
    }
    
    success_count = 0
    total_collections = len([k for k in transformed_data.keys() if k in collection_config])
    total_documents_loaded = 0
    
    print("\n" + "="*60)
    print("🚀 PROCESO DE CARGA MASIVA A MONGODB")
    print("="*60)
    print(f"📊 Total de colecciones a cargar: {total_collections}")
    print(f"⏱️ Iniciado: {start_time.strftime('%H:%M:%S')}")
    print("="*60)
    
    try:
        # Cargar cada dataset
        for dataset_num, (dataset_name, df) in enumerate(transformed_data.items(), 1):
            if dataset_name in collection_config:
                config = collection_config[dataset_name]
                collection_name = config['collection']
                batch_size = config['batch_size']
                
                print(f"\n📋 COLECCIÓN {dataset_num}/{total_collections}: {collection_name.upper()}")
                print(f"   📊 Registros a cargar: {len(df):,}")
                print(f"   📦 Tamaño de lote: {batch_size:,}")
                
                collection_start = datetime.now()
                
                if load_to_mongodb(db, collection_name, df, batch_size):
                    success_count += 1
                    total_documents_loaded += len(df)
                    
                    collection_duration = datetime.now() - collection_start
                    rate = len(df) / collection_duration.total_seconds() if collection_duration.total_seconds() > 0 else 0
                    
                    print(f"   ✅ Completada en {collection_duration.total_seconds():.1f}s "
                          f"({rate:.0f} docs/sec)")
                else:
                    print(f"   ❌ FALLÓ la carga de {dataset_name}")
                    logger.error(f"❌ Falló la carga de {dataset_name}")
        
        print("\n" + "="*60)
        
        # Crear índices después de cargar todos los datos
        if success_count == total_collections:
            print("🔧 CREANDO ÍNDICES PARA OPTIMIZAR CONSULTAS...")
            index_start = datetime.now()
            create_indexes(db)
            index_duration = datetime.now() - index_start
            print(f"✅ Índices creados en {index_duration.total_seconds():.1f}s")
            
        # Generar estadísticas finales detalladas
        print("\n📊 ESTADÍSTICAS FINALES DE CARGA:")
        print("-" * 50)
        
        grand_total = 0
        for dataset_name, config in collection_config.items():
            if dataset_name in transformed_data:
                collection_name = config['collection']
                try:
                    count = db[collection_name].count_documents({})
                    grand_total += count
                    status = "✅" if count > 0 else "❌"
                    print(f"{status} {collection_name:15}: {count:,} documentos")
                except Exception as e:
                    print(f"❌ {collection_name:15}: Error verificando - {str(e)}")
        
        print("-" * 50)
        print(f"📊 TOTAL DOCUMENTOS: {grand_total:,}")
        
        # Resumen final del proceso
        total_duration = datetime.now() - start_time
        overall_rate = total_documents_loaded / total_duration.total_seconds() if total_duration.total_seconds() > 0 else 0
        
        print(f"\n🎯 RESUMEN DEL PROCESO:")
        print(f"   ✅ Colecciones exitosas: {success_count}/{total_collections}")
        print(f"   📊 Documentos cargados: {total_documents_loaded:,}")
        print(f"   ⏱️ Tiempo total: {total_duration}")
        print(f"   🚀 Velocidad promedio: {overall_rate:.0f} docs/segundo")
        
        if success_count == total_collections:
            print(f"\n🎉 ¡PROCESO COMPLETADO EXITOSAMENTE!")
        else:
            print(f"\n⚠️ PROCESO PARCIALMENTE COMPLETADO")
        
        print("="*60)
        
        return success_count == total_collections
        
    except Exception as e:
        logger.error(f"❌ Error crítico en proceso de carga: {str(e)}")
        print(f"❌ Error crítico: {str(e)}")
        return False
        
    finally:
        # Cerrar conexión
        if client:
            client.close()
            logger.info("🔌 Conexión a MongoDB cerrada")

### 🚀 **Carga Completa con Retroalimentación Avanzada**

#### **📋 `load_all_data_to_mongodb_enhanced()` - Orquestador Principal**

Esta función coordina la carga completa de todos los datasets con características avanzadas:

**⚙️ Configuración adaptativa por colección:**
```python
collection_config = {
    'customers': {'collection': 'customers', 'batch_size': 500},     # Datos simples
    'orders': {'collection': 'orders', 'batch_size': 300},          # Muchas fechas
    'products': {'collection': 'products', 'batch_size': 500},      # Tamaño medio
    'order_items': {'collection': 'order_items', 'batch_size': 200}, # Dataset grande
    'payments': {'collection': 'payments', 'batch_size': 300},      # Datos financieros
    'reviews': {'collection': 'reviews', 'batch_size': 300},        # Texto variable
    'sellers': {'collection': 'sellers', 'batch_size': 1000},       # Datos simples
    'geolocation': {'collection': 'geolocation', 'batch_size': 100} # Dataset muy grande
}
```

**📊 Características principales:**

1. **Dashboard de progreso en tiempo real:**
   - Contador de colecciones (1/8, 2/8, etc.)
   - Progreso individual por colección
   - Métricas de velocidad (documentos/segundo)
   - Tiempo de procesamiento por colección

2. **Gestión inteligente de recursos:**
   - Tamaños de lote optimizados por tipo de datos
   - Conexión única reutilizada para todas las colecciones
   - Limpieza automática de memoria entre procesos

3. **Monitoreo y estadísticas:**
   - Tiempo de inicio y duración total
   - Velocidad promedio del proceso completo
   - Conteo final de documentos por colección
   - Verificación de integridad post-carga

4. **Post-procesamiento automático:**
   - Creación de índices después de la carga
   - Validación de conteos de documentos
   - Generación de reporte final detallado
   - Cierre seguro de conexiones

In [29]:
def run_complete_etl_pipeline():
    """
    Ejecuta el pipeline ETL completo con retroalimentación mejorada
    """
    start_time = datetime.now()
    logger.info("🚀 INICIANDO PROCESO ETL COMPLETO CON RETROALIMENTACIÓN MEJORADA")
    logger.info("="*70)
    
    try:
        # PASO 1: EXTRACCIÓN
        print("\n📥 FASE 1: EXTRACCIÓN DE DATOS")
        print("="*40)
        extraction_start = datetime.now()
        
        raw_data = extract_data()
        
        if not raw_data:
            logger.error("❌ Error en extracción de datos")
            return False
        
        extraction_duration = datetime.now() - extraction_start
        print(f"✅ Extracción completada en {extraction_duration.total_seconds():.1f}s")
        
        # PASO 2: TRANSFORMACIÓN
        print("\n🔧 FASE 2: TRANSFORMACIÓN DE DATOS")
        print("="*40)
        transformation_start = datetime.now()
        
        transformed_data = transform_all_data(raw_data)
        
        transformation_duration = datetime.now() - transformation_start
        print(f"✅ Transformación completada en {transformation_duration.total_seconds():.1f}s")
        
        # PASO 3: CARGA CON RETROALIMENTACIÓN MEJORADA
        print("\n📤 FASE 3: CARGA A MONGODB CON RETROALIMENTACIÓN MEJORADA")
        print("="*60)
        loading_start = datetime.now()
        
        # Usar la función de carga mejorada
        success = load_all_data_to_mongodb_enhanced(transformed_data)
        
        loading_duration = datetime.now() - loading_start
        print(f"\n⏱️ Carga completada en {loading_duration.total_seconds():.1f}s")
        
        # RESUMEN FINAL DETALLADO
        end_time = datetime.now()
        total_duration = end_time - start_time
        
        print("\n" + "="*70)
        print("📊 RESUMEN COMPLETO DEL PROCESO ETL")
        print("="*70)
        
        print(f"⏱️ TIEMPOS DE EJECUCIÓN:")
        print(f"   📥 Extracción:     {extraction_duration.total_seconds():6.1f}s")
        print(f"   🔧 Transformación: {transformation_duration.total_seconds():6.1f}s") 
        print(f"   📤 Carga:          {loading_duration.total_seconds():6.1f}s")
        print(f"   ⏱️ TOTAL:          {total_duration.total_seconds():6.1f}s")
        
        print(f"\n📅 TIMESTAMPS:")
        print(f"   🚀 Iniciado:   {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"   🏁 Finalizado: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
        
        if success:
            print(f"\n🎉 ¡PROCESO ETL COMPLETADO EXITOSAMENTE!")
            print(f"✅ Todos los datos están disponibles en MongoDB")
            print(f"🗄️ Base de datos: '{MONGODB_CONFIG['database']}'")
            print(f"🔗 Host: {MONGODB_CONFIG['host']}:{MONGODB_CONFIG['port']}")
        else:
            print(f"\n❌ PROCESO ETL FALLÓ")
            print(f"⚠️ Revisa los logs para más detalles")
            
        print("="*70)
        
        return success
        
    except Exception as e:
        logger.error(f"❌ Error crítico en pipeline ETL: {str(e)}")
        print(f"\n❌ ERROR CRÍTICO: {str(e)}")
        return False

# FUNCIÓN DE EJECUCIÓN RÁPIDA PARA PRUEBAS
def run_quick_etl_test(collections_to_test: List[str] = None):
    """
    Ejecuta una versión rápida del ETL para pruebas con colecciones específicas
    """
    if collections_to_test is None:
        collections_to_test = ['customers', 'orders']  # Solo unas pocas para prueba rápida
    
    print("🧪 EJECUTANDO ETL DE PRUEBA RÁPIDA")
    print("="*40)
    print(f"📋 Colecciones a probar: {', '.join(collections_to_test)}")
    
    try:
        # Extracción
        raw_data = extract_data()
        if not raw_data:
            return False
        
        # Transformación
        transformed_data = transform_all_data(raw_data)
        
        # Filtrar solo las colecciones de prueba
        test_data = {k: v for k, v in transformed_data.items() if k in collections_to_test}
        
        # Carga con lotes pequeños para prueba rápida
        success = load_all_data_to_mongodb_enhanced(test_data)
        
        if success:
            print("✅ Prueba ETL completada exitosamente")
        else:
            print("❌ Prueba ETL falló")
            
        return success
        
    except Exception as e:
        print(f"❌ Error en prueba ETL: {str(e)}")
        return False

# INFORMACIÓN PARA EL USUARIO
print("🛠️ FUNCIONES ETL MEJORADAS DISPONIBLES:")
print("="*50)
print("📋 Funciones principales:")
print("   • run_complete_etl_pipeline() - ETL completo con retroalimentación mejorada")
print("   • run_quick_etl_test() - ETL rápido para pruebas")
print("   • load_all_data_to_mongodb_enhanced() - Solo carga con mejor feedback")
print()
print("🔧 Mejoras implementadas:")
print("   ✅ Carga por lotes optimizada")
print("   ✅ Barra de progreso visual")
print("   ✅ Estadísticas en tiempo real")
print("   ✅ Manejo de errores por lote")
print("   ✅ Recuperación automática de errores")
print("   ✅ Métricas de velocidad de carga")
print("   ✅ Resumen detallado final")
print()
print("📝 Para ejecutar:")
print("   # ETL completo mejorado:")
print("   etl_success = run_complete_etl_pipeline()")
print()
print("   # Prueba rápida:")
print("   test_success = run_quick_etl_test(['customers', 'orders'])")

🛠️ FUNCIONES ETL MEJORADAS DISPONIBLES:
📋 Funciones principales:
   • run_complete_etl_pipeline() - ETL completo con retroalimentación mejorada
   • run_quick_etl_test() - ETL rápido para pruebas
   • load_all_data_to_mongodb_enhanced() - Solo carga con mejor feedback

🔧 Mejoras implementadas:
   ✅ Carga por lotes optimizada
   ✅ Barra de progreso visual
   ✅ Estadísticas en tiempo real
   ✅ Manejo de errores por lote
   ✅ Recuperación automática de errores
   ✅ Métricas de velocidad de carga
   ✅ Resumen detallado final

📝 Para ejecutar:
   # ETL completo mejorado:
   etl_success = run_complete_etl_pipeline()

   # Prueba rápida:
   test_success = run_quick_etl_test(['customers', 'orders'])


### 🎯 **Pipeline ETL Principal y Funciones de Control**

Esta sección contiene las funciones orquestadoras que coordinan todo el proceso ETL:

#### **🚀 `run_complete_etl_pipeline()` - Función Principal**

Ejecuta el proceso ETL completo de principio a fin con monitoreo avanzado:

**📋 Fases del proceso:**

1. **📥 FASE 1: EXTRACCIÓN**
   - Carga todos los archivos CSV del dataset Olist
   - Valida existencia y formato de archivos
   - Registra estadísticas de carga por archivo
   - Mide tiempo de extracción

2. **🔧 FASE 2: TRANSFORMACIÓN**
   - Aplica todas las transformaciones específicas por dataset
   - Limpia y estandariza datos
   - Crea campos derivados y métricas de negocio
   - Mide tiempo de transformación

3. **📤 FASE 3: CARGA MEJORADA**
   - Utiliza `load_all_data_to_mongodb_enhanced()`
   - Carga optimizada por lotes con retroalimentación
   - Creación automática de índices
   - Mide tiempo de carga

**📊 Métricas y reportes:**
- Tiempos detallados por fase
- Timestamps de inicio y finalización
- Resumen de éxito/fallo por componente
- Información de conexión a MongoDB

#### **🧪 `run_quick_etl_test()` - Función de Pruebas**

Versión ligera para validación y desarrollo:
- **Propósito**: Probar cambios sin procesar todos los datos
- **Por defecto**: Solo procesa 'customers' y 'orders'
- **Personalizable**: Permite especificar qué colecciones probar
- **Uso típico**: Validar configuraciones antes del ETL completo

**💡 Casos de uso:**
```python
# Prueba básica con 2 colecciones
test_success = run_quick_etl_test()

# Prueba personalizada con colecciones específicas
test_success = run_quick_etl_test(['products', 'payments', 'reviews'])
```

In [30]:
# 🚀 EJECUTAR ETL COMPLETO CON RETROALIMENTACIÓN MEJORADA
print("🚀 INICIANDO ETL MEJORADO CON MEJOR RETROALIMENTACIÓN")
print("="*60)

etl_success = run_complete_etl_pipeline()

2025-08-13 22:26:53,994 - INFO - 🚀 INICIANDO PROCESO ETL COMPLETO CON RETROALIMENTACIÓN MEJORADA
2025-08-13 22:26:53,995 - INFO - ======================================================================
2025-08-13 22:26:53,996 - INFO - 🔄 Iniciando extracción de datos...
2025-08-13 22:26:54,160 - INFO - ✅ customers: 99,441 filas cargadas


🚀 INICIANDO ETL MEJORADO CON MEJOR RETROALIMENTACIÓN

📥 FASE 1: EXTRACCIÓN DE DATOS


2025-08-13 22:26:54,896 - INFO - ✅ geolocation: 1,000,163 filas cargadas
2025-08-13 22:26:55,141 - INFO - ✅ order_items: 112,650 filas cargadas
2025-08-13 22:26:55,245 - INFO - ✅ order_payments: 103,886 filas cargadas
2025-08-13 22:26:55,606 - INFO - ✅ order_reviews: 99,224 filas cargadas
2025-08-13 22:26:55,986 - INFO - ✅ orders: 99,441 filas cargadas
2025-08-13 22:26:56,029 - INFO - ✅ products: 32,951 filas cargadas
2025-08-13 22:26:56,035 - INFO - ✅ sellers: 3,095 filas cargadas
2025-08-13 22:26:56,038 - INFO - ✅ product_categories: 71 filas cargadas
2025-08-13 22:26:56,039 - INFO - 📊 Extracción completada: 9 datasets, 1,550,922 filas totales
2025-08-13 22:26:56,040 - INFO - 🔄 Iniciando transformación completa de datos...
2025-08-13 22:26:56,040 - INFO - 🔄 Transformando datos de clientes...
2025-08-13 22:26:56,156 - INFO - ✅ Clientes transformados: 99,441 registros
2025-08-13 22:26:56,157 - INFO - 🔄 Transformando datos de órdenes...


✅ Extracción completada en 2.0s

🔧 FASE 2: TRANSFORMACIÓN DE DATOS


2025-08-13 22:26:57,624 - INFO - ✅ Órdenes transformadas: 99,441 registros
2025-08-13 22:26:57,625 - INFO - 🔄 Transformando datos de productos...
2025-08-13 22:26:57,639 - INFO - ✅ Productos transformados: 32,951 registros
2025-08-13 22:26:57,641 - INFO - 🔄 Transformando datos de items...
2025-08-13 22:26:57,654 - INFO - ✅ Items transformados: 112,650 registros
2025-08-13 22:26:57,655 - INFO - 🔄 Transformando datos de pagos...
2025-08-13 22:26:57,716 - INFO - ✅ Pagos transformados: 103,886 registros
2025-08-13 22:26:57,717 - INFO - 🔄 Transformando datos de reseñas...
2025-08-13 22:26:57,884 - INFO - ✅ Reseñas transformadas: 99,224 registros
2025-08-13 22:26:57,899 - INFO - ✅ Transformación completa finalizada
2025-08-13 22:26:57,900 - INFO - 🚀 Iniciando carga completa a MongoDB con retroalimentación mejorada...
2025-08-13 22:26:57,912 - INFO - ✅ Conexión exitosa a MongoDB: olist_ecommerce
2025-08-13 22:26:57,913 - INFO - 🔄 Iniciando carga de 99,441 registros a 'customers'...


...

   ✅ Carga completada para 'geolocation':
      📊 Documentos insertados: 1,000,163
      🔍 Documentos verificados: 1,000,163
      📦 Lotes procesados: 10002
      ❌ Lotes con errores: 0
   ✅ Completada en 39.2s (25544 docs/sec)

🔧 CREANDO ÍNDICES PARA OPTIMIZAR CONSULTAS...


2025-08-13 22:27:55,764 - INFO - 🔄 Creando índices para optimizar consultas...
2025-08-13 22:27:58,942 - INFO - ✅ Índices creados exitosamente


✅ Índices creados en 3.2s

📊 ESTADÍSTICAS FINALES DE CARGA:
--------------------------------------------------
✅ customers      : 99,441 documentos
✅ orders         : 99,441 documentos
✅ products       : 32,951 documentos
✅ order_items    : 112,650 documentos
✅ payments       : 103,886 documentos
✅ reviews        : 99,224 documentos
✅ sellers        : 3,095 documentos


2025-08-13 22:27:59,463 - INFO - 🔌 Conexión a MongoDB cerrada


✅ geolocation    : 1,000,163 documentos
--------------------------------------------------
📊 TOTAL DOCUMENTOS: 1,550,851

🎯 RESUMEN DEL PROCESO:
   ✅ Colecciones exitosas: 8/8
   📊 Documentos cargados: 1,550,851
   ⏱️ Tiempo total: 0:01:01.560159
   🚀 Velocidad promedio: 25192 docs/segundo

🎉 ¡PROCESO COMPLETADO EXITOSAMENTE!

⏱️ Carga completada en 61.6s

📊 RESUMEN COMPLETO DEL PROCESO ETL
⏱️ TIEMPOS DE EJECUCIÓN:
   📥 Extracción:        2.0s
   🔧 Transformación:    1.9s
   📤 Carga:            61.6s
   ⏱️ TOTAL:            65.5s

📅 TIMESTAMPS:
   🚀 Iniciado:   2025-08-13 22:26:53
   🏁 Finalizado: 2025-08-13 22:27:59

🎉 ¡PROCESO ETL COMPLETADO EXITOSAMENTE!
✅ Todos los datos están disponibles en MongoDB
🗄️ Base de datos: 'olist_ecommerce'
🔗 Host: localhost:27020


### ▶️ **Ejecución del ETL Completo**

**Esta celda ejecuta todo el proceso ETL mejorado:**

Al ejecutar `run_complete_etl_pipeline()` se inicia un proceso automatizado que:

1. **🔍 Verifica prerequisitos**: Conexión a MongoDB, archivos CSV disponibles
2. **📥 Extrae datos**: Carga todos los datasets del directorio `data/`
3. **🔧 Transforma datos**: Aplica limpieza, validaciones y campos derivados
4. **📤 Carga a MongoDB**: Inserción optimizada por lotes con retroalimentación
5. **🔧 Crea índices**: Optimiza la base de datos para consultas rápidas
6. **✅ Valida resultados**: Verifica integridad y genera reporte final

**⏱️ Tiempo estimado**: 2-5 minutos dependiendo del hardware y tamaño de datos

**📊 Output esperado**:
- Progreso detallado por fase
- Barras de progreso por colección
- Métricas de velocidad (docs/segundo)
- Estadísticas finales de carga
- Confirmación de éxito/fallo

**🚨 Prerequisitos importantes**:
- MongoDB debe estar ejecutándose en el puerto configurado (27020)
- Los archivos CSV deben estar en el directorio `data/`
- Suficiente espacio en disco para la base de datos

## 5. ✅ VALIDACIÓN Y CONSULTAS DE PRUEBA

Funciones para validar que los datos se cargaron correctamente y realizar consultas de prueba

In [31]:
def validate_mongodb_data():
    """
    Valida que los datos se cargaron correctamente en MongoDB
    """
    logger.info("🔍 Validando datos en MongoDB...")
    
    client, db = get_mongodb_connection()
    
    if db is None:
        logger.error("❌ No se pudo conectar a MongoDB para validación")
        return False
    
    try:
        collections = ['customers', 'orders', 'products', 'order_items', 
                      'payments', 'reviews', 'sellers', 'geolocation']
        
        print("📊 ESTADÍSTICAS DE DATOS EN MONGODB")
        print("="*50)
        
        total_documents = 0
        
        for collection_name in collections:
            collection = db[collection_name]
            count = collection.count_documents({})
            total_documents += count
            
            # Obtener un documento de ejemplo
            sample_doc = collection.find_one()
            fields_count = len(sample_doc.keys()) if sample_doc else 0
            
            print(f"📋 {collection_name}:")
            print(f"   📄 Documentos: {count:,}")
            print(f"   🏷️ Campos: {fields_count}")
            print()
        
        print(f"📊 TOTAL DE DOCUMENTOS: {total_documents:,}")
        
        # Validaciones específicas
        print("\\n🔍 VALIDACIONES ESPECÍFICAS:")
        
        # 1. Verificar que existen órdenes con estado 'delivered'
        delivered_orders = db.orders.count_documents({'order_status': 'delivered'})
        print(f"✅ Órdenes entregadas: {delivered_orders:,}")
        
        # 2. Verificar rangos de fechas
        date_range = db.orders.aggregate([
            {'$group': {
                '_id': None,
                'min_date': {'$min': '$order_purchase_timestamp'},
                'max_date': {'$max': '$order_purchase_timestamp'}
            }}
        ])
        
        for doc in date_range:
            print(f"📅 Rango de fechas: {doc['min_date'].strftime('%Y-%m-%d')} a {doc['max_date'].strftime('%Y-%m-%d')}")
        
        # 3. Verificar integridad referencial básica
        order_count = db.orders.count_documents({})
        payment_count = db.payments.count_documents({})
        print(f"🔗 Órdenes: {order_count:,}, Pagos: {payment_count:,}")
        
        logger.info("✅ Validación completada exitosamente")
        return True
        
    except Exception as e:
        logger.error(f"❌ Error en validación: {str(e)}")
        return False
    
    finally:
        if client:
            client.close()

def run_sample_queries():
    """
    Ejecuta consultas de ejemplo para demostrar funcionalidad
    """
    logger.info("🔍 Ejecutando consultas de ejemplo...")
    
    client, db = get_mongodb_connection()
    
    if db is None:
        logger.error("❌ No se pudo conectar a MongoDB")
        return
    
    try:
        print("🔍 CONSULTAS DE EJEMPLO EN MONGODB")
        print("="*50)
        
        # 1. Top 5 estados con más clientes
        print("1️⃣ Top 5 Estados con Más Clientes:")
        top_states = db.customers.aggregate([
            {'$group': {'_id': '$customer_state', 'count': {'$sum': 1}}},
            {'$sort': {'count': -1}},
            {'$limit': 5}
        ])
        
        for state in top_states:
            print(f"   {state['_id']}: {state['count']:,} clientes")
        
        # 2. Promedio de satisfacción por mes
        print("\\n2️⃣ Promedio de Satisfacción por Mes (últimos 6 meses):")
        monthly_satisfaction = db.reviews.aggregate([
            {'$match': {'review_score': {'$exists': True}}},
            {'$group': {
                '_id': {
                    'year': {'$year': '$review_creation_date'},
                    'month': {'$month': '$review_creation_date'}
                },
                'avg_score': {'$avg': '$review_score'},
                'count': {'$sum': 1}
            }},
            {'$sort': {'_id.year': -1, '_id.month': -1}},
            {'$limit': 6}
        ])
        
        for month in monthly_satisfaction:
            year = month['_id']['year']
            month_num = month['_id']['month']
            avg_score = month['avg_score']
            count = month['count']
            print(f"   {year}-{month_num:02d}: {avg_score:.2f} ⭐ ({count} reviews)")
        
        # 3. Top 5 categorías por revenue
        print("\\n3️⃣ Top 5 Categorías por Revenue:")
        category_revenue = db.order_items.aggregate([
            {'$lookup': {
                'from': 'products',
                'localField': 'product_id',
                'foreignField': 'product_id',
                'as': 'product_info'
            }},
            {'$unwind': '$product_info'},
            {'$group': {
                '_id': '$product_info.product_category_name_english',
                'total_revenue': {'$sum': '$price'},
                'total_orders': {'$sum': 1}
            }},
            {'$sort': {'total_revenue': -1}},
            {'$limit': 5}
        ])
        
        for category in category_revenue:
            name = category['_id'] or 'Unknown'
            revenue = category['total_revenue']
            orders = category['total_orders']
            print(f"   {name}: R$ {revenue:,.2f} ({orders:,} items)")
        
        # 4. Distribución de métodos de pago
        print("\\n4️⃣ Distribución de Métodos de Pago:")
        payment_distribution = db.payments.aggregate([
            {'$group': {
                '_id': '$payment_type',
                'count': {'$sum': 1},
                'total_value': {'$sum': '$payment_value'}
            }},
            {'$sort': {'count': -1}}
        ])
        
        for payment in payment_distribution:
            method = payment['_id']
            count = payment['count']
            value = payment['total_value']
            print(f"   {method}: {count:,} transacciones (R$ {value:,.2f})")
        
        logger.info("✅ Consultas de ejemplo completadas")
        
    except Exception as e:
        logger.error(f"❌ Error ejecutando consultas: {str(e)}")
    
    finally:
        if client:
            client.close()

# Funciones de utilidad para consultas
print("🛠️ FUNCIONES DE VALIDACIÓN Y CONSULTAS LISTAS")
print("📝 Para usar:")
print("   - validate_mongodb_data() : Valida los datos cargados")
print("   - run_sample_queries() : Ejecuta consultas de ejemplo")

🛠️ FUNCIONES DE VALIDACIÓN Y CONSULTAS LISTAS
📝 Para usar:
   - validate_mongodb_data() : Valida los datos cargados
   - run_sample_queries() : Ejecuta consultas de ejemplo


### ✅ **Funciones de Validación y Consultas de Ejemplo**

Esta sección proporciona herramientas para validar y explorar los datos cargados en MongoDB:

#### **🔍 `validate_mongodb_data()` - Validación de Integridad**

Función completa de validación post-ETL que verifica:

**📊 Estadísticas básicas:**
- Conteo de documentos por colección
- Número de campos por tipo de documento
- Total de documentos en la base de datos

**✅ Validaciones específicas:**
- **Órdenes entregadas**: Verifica que existen órdenes con status 'delivered'
- **Rangos de fechas**: Valida que las fechas están en rangos esperados
- **Integridad referencial**: Compara conteos entre colecciones relacionadas
- **Consistencia de datos**: Verifica que no hay datos corruptos

#### **🔍 `run_sample_queries()` - Consultas Demostrativas**

Ejecuta consultas representativas para demostrar la funcionalidad de MongoDB:

**📈 Consultas implementadas:**

1. **Top 5 Estados por Clientes**: Análisis geográfico de la base de clientes
2. **Satisfacción Temporal**: Promedio de satisfacción por mes (últimos 6 meses)
3. **Revenue por Categoría**: Top 5 categorías de productos por ingresos
4. **Métodos de Pago**: Distribución de tipos de pago y valores

**🎯 Propósito de las consultas:**
- **Validar joins**: Verificar que las relaciones entre colecciones funcionan
- **Probar agregaciones**: Confirmar que MongoDB puede realizar cálculos complejos
- **Demostrar capacidades**: Mostrar el tipo de análisis posibles
- **Benchmark**: Medir performance de consultas con índices

**📝 Uso recomendado:**
```python
# Después del ETL, validar la carga
validate_mongodb_data()

# Explorar datos con consultas de ejemplo
run_sample_queries()
```

In [34]:
validate_mongodb_data()

2025-08-13 22:32:32,291 - INFO - 🔍 Validando datos en MongoDB...
2025-08-13 22:32:32,315 - INFO - ✅ Conexión exitosa a MongoDB: olist_ecommerce


📊 ESTADÍSTICAS DE DATOS EN MONGODB
📋 customers:
   📄 Documentos: 99,441
   🏷️ Campos: 7

📋 orders:
   📄 Documentos: 99,441
   🏷️ Campos: 16

📋 products:
   📄 Documentos: 32,951
   🏷️ Campos: 13

📋 order_items:
   📄 Documentos: 112,650
   🏷️ Campos: 11

📋 payments:
   📄 Documentos: 103,886
   🏷️ Campos: 7

📋 reviews:
   📄 Documentos: 99,224
   🏷️ Campos: 12

📋 sellers:
   📄 Documentos: 3,095
   🏷️ Campos: 5



2025-08-13 22:32:33,014 - INFO - ✅ Validación completada exitosamente


📋 geolocation:
   📄 Documentos: 1,000,163
   🏷️ Campos: 6

📊 TOTAL DE DOCUMENTOS: 1,550,851
\n🔍 VALIDACIONES ESPECÍFICAS:
✅ Órdenes entregadas: 96,478
📅 Rango de fechas: 2016-09-04 a 2018-10-17
🔗 Órdenes: 99,441, Pagos: 103,886


True

In [35]:
run_sample_queries()

2025-08-13 22:32:43,796 - INFO - 🔍 Ejecutando consultas de ejemplo...
2025-08-13 22:32:43,807 - INFO - ✅ Conexión exitosa a MongoDB: olist_ecommerce


🔍 CONSULTAS DE EJEMPLO EN MONGODB
1️⃣ Top 5 Estados con Más Clientes:
   SP: 41,746 clientes
   RJ: 12,852 clientes
   MG: 11,635 clientes
   RS: 5,466 clientes
   PR: 5,045 clientes
\n2️⃣ Promedio de Satisfacción por Mes (últimos 6 meses):
   2018-08: 4.21 ⭐ (8987 reviews)
   2018-07: 4.29 ⭐ (5634 reviews)
   2018-06: 4.20 ⭐ (6715 reviews)
   2018-05: 4.19 ⭐ (7458 reviews)
   2018-04: 3.92 ⭐ (7287 reviews)
   2018-03: 3.73 ⭐ (7803 reviews)
\n3️⃣ Top 5 Categorías por Revenue:


KeyboardInterrupt: 